In [1]:
import sqlalchemy
from bs4 import BeautifulSoup
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.types import *
from sqlalchemy import create_engine
import os
import json
import time

In [6]:
spark = SparkSession.builder \
    .appName("Soccer Data") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [11]:
# def fetch_matchday(self):
log_name = "matchday"
url = f"https://www.sofascore.com/api/v1/sport/football/scheduled-events/2023-07-31"
response = requests.get(url)
status_code = response.status_code
if status_code != 200:
    print(f"Error code: {status_code}. Stop jobs!")
else:
    match_in_day = response.json()
    match_in_day = match_in_day['events']

    df_matchday = pd.json_normalize(match_in_day)
    df_matchday.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
    df_matchday.rename(columns={"id": "match_id"}, inplace=True)
    df_football = df_matchday[df_matchday['tournament_category_sport_name'] == 'Football']

    df = pd.concat([df_football], ignore_index=True)
    print("Dataframe is here!!!")
    print(df)
    # self.export(log_name, df)
    list_match_id = df["match_id"].astype(str).tolist()
    list_customId = df["customId"]
    return list_match_id, list_customId

# def shotmap(self, list_match_id):
#     shotmap_data = []
# 
#     for match_id in list_match_id:
#         url = f"https://www.sofascore.com/api/v1/event/{match_id}/shotmap"
#         try:
#             response = requests.get(url)
#             status_code = response.status_code
#             if status_code == 200:
#                 data = response.json()
#                 for event in data:
#                     event['match_id'] = match_id  # Add match_id to each event
#                 shotmap_data.extend(data)
#             else:
#                 print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
#         except Exception as e:
#             print(f"An error occurred for match ID {match_id}: {e}")
#         time.sleep(1)
# 
#     if shotmap_data:
#         # Normalize the nested JSON data
#         df_shotmap = pd.json_normalize(shotmap_data)
#         df_shotmap.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
#         return df_shotmap
#     else:
#         return None

SyntaxError: 'return' outside function (3972095364.py, line 23)

In [8]:
print(list_match_id)

['10834018', '10834019', '10834021', '10834020', '10834022', '10834023', '10834024', '10834025', '11067597', '11067611', '11067595', '11067601', '11369787', '11369789', '11369790', '10981560', '10981558', '10981625', '10981569', '10981584', '11347285', '11347286', '11471485', '11471490', '11471489', '11471492', '11076542', '11076538', '11076534', '11423813', '11423815', '11423812', '11423814', '11423817', '11423819', '11423816', '11423818', '11156282', '11156265', '11156267', '11156304', '11156301', '11156291', '11156298', '11156296', '11156286', '11156287', '11387604', '10961718', '10961715', '10961719', '10961720', '11434969', '11434971', '11434973', '11434975', '11434977', '11434965', '11434979', '11367610', '11367607', '11367611', '11367613', '11351393', '11351390', '11351391', '11351387', '11351389', '11369784', '11369786', '11369788', '11370322', '11200807', '11200803', '11200818', '11200806', '10932748', '10932743', '10932753', '10932749', '10932740', '10932758', '11464258', '11

In [9]:
# list_match_id = ["12173508"]
shotmap_data = []
for match_id in list_match_id:
    url = f"https://www.sofascore.com/api/v1/event/{match_id}/shotmap"
    try:
        response = requests.get(url)
        status_code = response.status_code
        if status_code == 200:
            data = response.json()
            for event in data['shotmap']:
                event_data = event.copy()
                event_data['match_id'] = match_id
                event_data['status_code'] = status_code
                shotmap_data.append(event_data)
        else:
            shotmap_data.append({'match_id': match_id, 'status_code': status_code})
            print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
    except Exception as e:
        shotmap_data.append({'match_id': match_id, 'status_code': None})
        print(f"An error occurred for match ID {match_id}: {e}")
    time.sleep(1)

if shotmap_data:
    df_shotmap = pd.json_normalize(shotmap_data)
    df_shotmap.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
    print(df_shotmap)
else:
    print("No data fetched.")
# if df_shotmap is not None:
#     self.export("shotmap", df_shotmap)
# else:
#     pass

# print(df_match_statistics)
# if match_summary_data:
#     df_summary = pd.json_normalize(match_summary_data, sep='_')
#     df_summary.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
#     print(df_summary)
# else:
#     print("No data fetched.")

Failed to fetch data for match ID 11434969. Status code: 404
Failed to fetch data for match ID 11351387. Status code: 404
Failed to fetch data for match ID 11351389. Status code: 404
Failed to fetch data for match ID 10997543. Status code: 404
Failed to fetch data for match ID 10997550. Status code: 404
Failed to fetch data for match ID 11000166. Status code: 404
Failed to fetch data for match ID 11000174. Status code: 404
Failed to fetch data for match ID 11000173. Status code: 404
Failed to fetch data for match ID 11000172. Status code: 404
Failed to fetch data for match ID 11000179. Status code: 404
Failed to fetch data for match ID 11471480. Status code: 404
Failed to fetch data for match ID 11404252. Status code: 404
Failed to fetch data for match ID 11404246. Status code: 404
Failed to fetch data for match ID 11404241. Status code: 404
Failed to fetch data for match ID 11114400. Status code: 404
Failed to fetch data for match ID 11114406. Status code: 404
Failed to fetch data for

In [10]:
print(df_shotmap)

     isHome shotType  situation    bodyPart goalMouthLocation         id  \
0     False     save    regular  right-foot        low-centre  2121744.0   
1      True     goal    penalty   left-foot          low-left  2121743.0   
2     False     save    regular  right-foot        high-right  2121740.0   
3     False     miss   assisted        head        close-left  2121739.0   
4      True    block   assisted  right-foot        low-centre  2121738.0   
...     ...      ...        ...         ...               ...        ...   
3296  False     miss  free-kick  right-foot        close-high  2122612.0   
3297  False     post   assisted   left-foot             right  2122611.0   
3298  False     save     corner   left-foot        low-centre  2122610.0   
3299  False    block   assisted  right-foot         low-right  2122609.0   
3300    NaN      NaN        NaN         NaN               NaN        NaN   

      time  addedTime  timeSeconds  reversedPeriodTime  ...  draw_end_y  \
0     84.0  

In [13]:
columns = df_match_event.columns.tolist()
print("Columns in df_match_event:")
for col in columns:
    print(col)

Columns in df_match_event:
match_id
status_code
customId
winnerCode
aggregatedWinnerCode
hasGlobalHighlights
hasEventPlayerStatistics
hasEventPlayerHeatMap
detailId
crowdsourcingDataDisplayEnabled
id
crowdsourcingEnabled
previousLegEventId
startTimestamp
slug
finalResultOnly
feedLocked
isEditor
tournament_name
tournament_slug
tournament_category_name
tournament_category_slug
tournament_category_sport_name
tournament_category_sport_slug
tournament_category_sport_id
tournament_category_id
tournament_category_country_alpha2
tournament_category_country_alpha3
tournament_category_country_name
tournament_category_flag
tournament_category_alpha2
tournament_uniqueTournament_name
tournament_uniqueTournament_slug
tournament_uniqueTournament_primaryColorHex
tournament_uniqueTournament_secondaryColorHex
tournament_uniqueTournament_category_name
tournament_uniqueTournament_category_slug
tournament_uniqueTournament_category_sport_name
tournament_uniqueTournament_category_sport_slug
tournament_unique

In [31]:
print(combined_df[combined_df["status_code"] == 404])

Empty DataFrame
Columns: [customId, winnerCode, hasGlobalHighlights, hasEventPlayerStatistics, hasEventPlayerHeatMap, detailId, crowdsourcingDataDisplayEnabled, match_id, crowdsourcingEnabled, startTimestamp, slug, finalResultOnly, feedLocked, isEditor, tournament_name, tournament_slug, tournament_category_name, tournament_category_slug, tournament_category_sport_name, tournament_category_sport_slug, tournament_category_sport_id, tournament_category_id, tournament_category_flag, tournament_uniqueTournament_name, tournament_uniqueTournament_slug, tournament_uniqueTournament_category_name, tournament_uniqueTournament_category_slug, tournament_uniqueTournament_category_sport_name, tournament_uniqueTournament_category_sport_slug, tournament_uniqueTournament_category_sport_id, tournament_uniqueTournament_category_id, tournament_uniqueTournament_category_flag, tournament_uniqueTournament_userCount, tournament_uniqueTournament_crowdsourcingEnabled, tournament_uniqueTournament_hasPerformanceGr

In [6]:
# customId = "xdbsEgb"
url = f"https://www.sofascore.com/api/v1/team/*"

response = requests.get(url)
status_code = response.status_code
print(status_code)
    

404


In [47]:
match_id = "12173508"
url = f"https://www.sofascore.com/api/v1/event/{match_id}/statistics"
try:
    response = requests.get(url)
    status_code = response.status_code
    if status_code == 200:
        data = response.json()
        # Assuming 'statistics' is a list containing match statistics
        for event in data['statistics']:
            for group in event['groups']:
                for stat_item in group['statisticsItems']:
                    # Append extracted data to the match_statistics_data list
                    match_statistics_data.append({
                        'period': event['period'],
                        'groupName': group['groupName'],
                        'name': stat_item['name'],
                        'home': stat_item['home'],
                        'away': stat_item['away'],
                        'compareCode': stat_item['compareCode'],
                        'statisticsType': stat_item['statisticsType'],
                        'valueType': stat_item['valueType'],
                        'homeValue': stat_item['homeValue'],
                        'awayValue': stat_item['awayValue'],
                        'renderType': stat_item['renderType'],
                        'key': stat_item['key']
                    })
    else:
        # Print an error message if the request fails
        print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
except requests.exceptions.RequestException as e:
    # Handle specific exceptions related to requests
    print(f"An error occurred for match ID {match_id}: {e}")

In [15]:
url = f"https://www.sofascore.com/api/v1/event/12173508/best-players/summary"
response = requests.get(url)
status_code = response.status_code
print(status_code)

200


In [19]:
df=pd.json_normalize(data)
print(df)

   confirmed                                       home.players  \
0       True  [{'player': {'name': 'Daniel Carvajal', 'slug'...   

  home.supportStaff home.formation home.playerColor.primary  \
0                []          4-4-2                   ffffff   

  home.playerColor.number home.playerColor.outline  \
0                  000000                   ffffff   

  home.playerColor.fancyNumber home.goalkeeperColor.primary  \
0                       222226                       4be438   

  home.goalkeeperColor.number  ... away.formation away.playerColor.primary  \
0                      ffffff  ...        4-2-3-1                   090606   

  away.playerColor.number away.playerColor.outline  \
0                  8d287e                   090606   

  away.playerColor.fancyNumber away.goalkeeperColor.primary  \
0                       ffffff                       f92b91   

  away.goalkeeperColor.number away.goalkeeperColor.outline  \
0                      cff149                  

In [18]:
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'host.docker.internal'
USER = 'postgres'
PASSWORD = 'ducnm7'
PORT = 5432
DATABASE = 'postgres'

date='2023-07-31'
connection_string = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

query = """
        SELECT matchday.match_id, matchday.custom_id 
        FROM matchday 
        WHERE ds = %s
        """
with engine.connect() as connection:
    df = pd.read_sql_query(query, connection, params=[date])

ArgumentError: List argument must consist only of tuples or dictionaries

In [13]:
df = pd.read_sql_table("matchday", engine)
print(df)

     customId  winnerCode  hasGlobalHighlights hasEventPlayerStatistics  \
0     nYcspYc         2.0                False                     True   
1    oYcsYDwb         2.0                False                     True   
2     zYcsCPi         3.0                False                     True   
3    JZmsijdb         2.0                False                     True   
4    lYcsaEwb         3.0                False                     True   
..        ...         ...                  ...                      ...   
359   GoksUjn         1.0                False                     None   
360    EPsTkk         3.0                False                     None   
361   ZTqsoGB         2.0                False                     None   
362    UHsXdb         2.0                False                     True   
363  KrcsBHtb         3.0                False                     None   

    hasEventPlayerHeatMap  detailId  crowdsourcingDataDisplayEnabled  \
0                    True  

In [40]:
url = "https://www.sofascore.com/newcastle-united-manchester-united/DYBe#12240571"
response = requests.get(url)